In [ ]:
!wget https://snap.stanford.edu/data/cit-HepTh-abstracts.tar.gz
!tar -xf cit-HepTh-abstracts.tar.gz

--2021-09-08 17:40:42--  https://snap.stanford.edu/data/cit-HepTh-abstracts.tar.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11956788 (11M) [application/x-gzip]
Saving to: ‘cit-HepTh-abstracts.tar.gz’

cit-HepTh-abstracts 100%[===================>]  11.40M   324KB/s    in 22s     

2021-09-08 17:41:04 (529 KB/s) - ‘cit-HepTh-abstracts.tar.gz’ saved [11956788/11956788]



### Ссылки:
- Датасет: https://snap.stanford.edu/data/cit-HepTh.html
- Датасет с журналами: https://www.scimagojr.com/journalrank.php
- Абривиатура в оригинал: https://images.webofknowledge.com/images/help/WOS/0-9_abrvjt.html


In [ ]:
from pathlib import Path
from tqdm.notebook import tqdm

In [ ]:
year = 1992
abstart_files = []
has_journal = 0
count = 0
existing_journals = []
while year < 2004:
  for file in list(Path(str(year)).iterdir()):
    count += 1
    abs = open(file, 'r').read()
    # print(abs)
    idx = abs.find('Journal-ref: ')
    if idx != -1:
      has_journal+=1

      journals = abs[idx+13: abs.find('\\', idx)-1].split(';')
      journals = map(lambda x: ''.join(x.split()[:-3]), journals)
      
      for jounral in journals:
        if jounral not in existing_journals:
          existing_journals.append(jounral)
      # print(journal)
  year += 1

for journal in existing_journals:
  print(journal)
print(len(existing_journals), has_journal, count, has_journal / count)

Mod.Phys.Lett.
J.Geom.Phys.
Nucl.Phys.
Erratum-ibid.
Phys.Rev.
Int.J.Mod.Phys.
J.Math.Phys.
Phys.Lett.
Gen.Rel.Grav.
Commun.Math.Phys.
Phys.Rev.Lett.
NuovoCim.
Theor.Math.Phys.
Class.Quant.Grav.
Z.Phys.
Lett.Math.Phys.
Prog.Theor.Phys.Suppl.
J.Phys.
Prog.Theor.Phys.
Teor.Mat.Fiz.
Comput.Phys.Commun.
Adv.Math.
Czech.J.Phys.
CommentsNucl.Part.Phys.
OnKluader'sPath:AFieldTrip,ed.G.G.Emch,G.C.Hegerfeldt,andL.Streit(World
Helv.Phys.Acta
AnnalsPhys.
Europhys.Lett.
in:StringquantumgravityandphysicsatthePlanckscale,erice,21-28June1992,ed.N.Sachez,World
Mod.Phys.Lett.A8(1993)923-936Subj-class:HighEnergy

Phys.Rept.
DukeMath.J.
Ann.Phys.
PacificJ.Math.
Int.J.Theor.Phys.
AnnalesPoincarePhys.Theor.
J.Statist.Phys.
Nucl.Phys.Proc.Suppl.
Astropart.Phys.
Prog.Math.
J.Math.Sci.
Zap.Nauchn.Semin.
SpringerLectureNotesinPhysics
Addendum-ibid.
inGroupTheoreticalmethodsinPhysics,Volume2,eds.M.A.delOlmo,M.SantanderandJ.M.Guilarte,
ActaPhys.Polon.
in:Fromsuperstringstosupergravity,Proc.of26thEloisatronWorksh

In [ ]:
import requests
import pickle

In [ ]:
abr_to_orig = pickle.load(open("abrs.pkl", 'rb'))

FileNotFoundError: ignored

In [ ]:
abr_to_orig2 = {}
for key, value in abr_to_orig.items():
   abr_to_orig2[key.replace(" ", '').replace(".", '').lower()] = value


In [ ]:
def get_orig(abr: str):
  abr = abr.replace(" ", '').replace(".", '').lower()
  return abr_to_orig2.get(abr)
get_orig('Ann.Phys.')

'Annals of Physics'

In [ ]:
ok = 0
for journal in existing_journals:
  if get_orig(journal):
    ok += 1
    print(get_orig(journal))
  else:
    print(journal)
ok, len(existing_journals)

Mod.Phys.Lett.
Journal of Geometric Analysis
Nucl.Phys.
Erratum-ibid.
Phys.Rev.
Int.J.Mod.Phys.
Journal of Mathematics and Physics
Phys.Lett.
Gen.Rel.Grav.
Communications Materials
Physical Review Fluids
NuovoCim.
Theoretical Issues in Ergonomics Science
Class.Quant.Grav.
Z.Phys.
Letters in Heat and Mass Transfer
Progress of Theoretical Physics
Journal of Phycology
Progress of Theoretical and Experimental Physics
Teor.Mat.Fiz.
Computers in Physics
Advanced Materials Technologies
Czechoslovak Journal of Food Sciences
CommentsNucl.Part.Phys.
OnKluader'sPath:AFieldTrip,ed.G.G.Emch,G.C.Hegerfeldt,andL.Streit(World
Helvetica Chimica Acta
AnnalsPhys.
EURO Journal on Transportation and Logistics
in:StringquantumgravityandphysicsatthePlanckscale,erice,21-28June1992,ed.N.Sachez,World
Mod.Phys.Lett.A8(1993)923-936Subj-class:HighEnergy

Phys.Rept.
Deutsche Zeitschrift fur Sportmedizin
Annals of Physics
PacificJ.Math.
International Journal of Technology, Policy and Management
AnnalesPoincarePhys.T

(74, 420)

In [ ]:
print(abr_to_orig2.keys())

dict_keys(['3biotech', '3dprintadditmanuf', '3dprintmed', 'a&a', 'aapgbull', 'aapgmem', 'aapsj', 'aatccjres', 'aatccrev', 'abbrev', 'abhakadwissddr', 'abhakadwissddr,abtmath,naturwiss,tech', 'abhmathseminunivhamburg', 'abhsenckbnaturforschges', 'aboricj', 'acatrans', 'acadforensicpathol', 'acadradiol', 'accchemres', 'accmaterres', 'accessmicrobiol', 'accidanalprev', 'accreditqualassur', 'ach-modelschem', 'achievlifesci', 'acimaterj', 'acistructj', 'acmcomputsurv', 'acmdigitalthreats:respract', 'acminroads', 'acmjcomputcultheritage', 'acmjdatainfqual', 'acmjemergingtechnolcomput', 'acmjemergingtechnolcomputsyst', 'acmjexpalgorithmics', 'acmqueue', 'acmtransaccessiblecomput', 'acmtransalgorithms', 'acmtransapplpercept', 'acmtransarchitcodeoptim', 'acmtransasianlanginfprocess', 'acmtransasianlow-resourlanginfprocess', 'acmtransautonadaptsyst', 'acmtranscomputeduc', 'acmtranscomputhealthcare', 'acmtranscomputlogic', 'acmtranscomputsyst', 'acmtranscomputtheory', 'acmtranscomput-huminteract'